# Additive Gaussian Processes

$$\begin{align}
k_{\text{add}}(x, x') = \sum_{d = 1}^D \sigma^2_d k_{\text{add}, d}(x, x')
\end{align}$$

$$\begin{align}
k_{\text{add}, d}(x, x') &= \sum_{1 \leq i_1 < ... < i_d \leq D} \prod_{n = 1}^N k_{i_n}(x_{i_n}, x_{i_n}') \\
&= \sum_{1 \leq i_1 < ... < i_d \leq D} \prod_{n = 1}^N z_{i_n}
\end{align}$$

By the Newton-Girard formula

$$\begin{align}
f_K(z_1, ..., z_D) &= \sum_{1 \leq i_1 < ... < i_d \leq K} \prod_{n = 1}^N z_{i_n} \\
&= \frac{1}{n} \sum_{k = 1}^K  (-1)^{k - 1} f_{K - k}(z_1, ..., z_D) s_k(z_1, ..., z_D)
\end{align}$$

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import HTML, set_matplotlib_formats
set_matplotlib_formats('pdf', 'svg')
css_style = open('../../../_static/custom_style.css', 'r').read()
HTML(f'<style>{css_style}</style>')

<div class="lemma">
    
**Definition (Newton-Girard formulae)** Let $z_1, ..., z_D \in \mathbb{R}$, $s_k$ be the power sum
    
$$\begin{align}
p_K(z_1, ..., z_D) = \sum_{d = 1}^D z_d^k.
\end{align}$$
    
and $s_K$ be the symmetric polynomial of order $K < D$, written as

$$\begin{align}
s_K(z_1, ..., z_D) = \sum_{1 \leq i_1 < ... < i_K \leq K} \prod_{k = 1}^K z_{i_k}.
\end{align}$$
    
The following recursive relation, known as the Newton-Girard formula, holds
    
$$\begin{align}
s_K(z_1, ..., z_D) = \frac{1}{N} \sum_{k = 1}^K  (-1)^{k - 1} s_{K - k}(z_1, ..., z_D) p_k(z_1, ..., z_D),
\end{align}$$
 
where we have defined $u_0 = 1$.
    
</div>
<br>

<details class="proof" open>
<summary>Derivation: Newton-Girard formulae</summary>
    
Starting from the polynomial
    
$$\begin{align}
f(x) &= \prod_{d = 1}^D (1 - z_d x)
\end{align}$$
    
and taking its derivative, we obtain
    
$$\begin{align}
f'(x) &= - f(x)\sum_{d = 1}^D \frac{z_d}{1 - z_d x} \\
      &= - f(x)\sum_{d = 1}^D \sum_{l = 0}^\infty z_d^{l + 1} x^l \\
      &= - f(x)\sum_{l = 0}^\infty p_{l + 1} x^l
\end{align}$$
    
Now, noting that the polynomial can also be written as
   
$$\begin{align}
f(x) &=  \sum_{d = 0}^D s_d(z_1, ..., z_D)~x^d \\
     &=  \sum_{d = 0}^D S_d~x^d,
\end{align}$$
    
we take a derivative of this expression to obtain
    
$$\begin{align}
f'(x) &= S_1 + 2S_2 x + ... + d S_d x^{d-1}.
\end{align}$$
    
    
Now equating the two expressions for the derivatives and rearranging to compare coefficients
    
$$\begin{align}
S_1 + 2S_2 x + ... + d S_d x^{d-1} &= - \left[\sum_{d = 0}^D S_d~x^d \right] \left[\sum_{l = 0}^\infty p_{l + 1} x^l\right] \\
                                     &= - \left[\sum_{d = 0}^D \sum_{l = 0}^\infty S_d p_{l + 1} x^{d + l}\right]
\end{align}$$
    
we obtain
    
$$\begin{align}
k S_k &= - \sum_{d = 0}^{k - 1} S_d p_{D - d}
\end{align}$$

</details>
<br>